---
## <center> **손쉽게 따라하는 SAS® Viya® Workbench - Python 사용자편** </center>
---

### **0. Packages 설치**

[주의] saspy 패키지를 설치한 뒤, 창을 다시 열어주세요.

In [ ]:
!pip install packages saspy

In [1]:
# SAS 연결
import saspy

# 데이터 처리
import numpy as np
import pandas as pd

# SAS Viya 알고리즘
from sasviya.ml.linear_model import LogisticRegression
from sasviya.ml.svm import SVC
from sasviya.ml.tree import DecisionTreeClassifier, ForestClassifier, GradientBoostingClassifier

# 모형 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, auc

# 그래프 생성
import matplotlib.pyplot as plt

**※ saspy 패키지**
- 개요
    - Python과 SAS 간의 인터페이스를 제공
    - Python 프로세스를 SAS 배포에 연결하여 SAS 코드를 실행
    - SAS 데이터 세트와 Pandas 데이터 프레임 간 데이터 표현 변환 지원
- 필수 조건
    - Python3.x 이상
    - SAS 9.4 이상
    - SAS Viya 3 이상

### **1. SAS 연결 및 Library 할당**

In [ ]:
# 세션 연결
sas = saspy.SASsession()

# Library 할당
sas.saslib('WRKLIB', path='-라이브러리 경로-')

### **2. 데이터 로드**

In [ ]:
# 형식 변환 (sas7bdat → Dataframe) 후 로드
churn_parted = sas.sasdata('churn_parted', libref='WRKLIB').to_df()
churn_parted.columns = churn_parted.columns.str.upper()
churn_parted.head(10)

### **3. 데이터 분할**  
- 학습(Training), 평가(Validation), 검증(Testing) 데이터 분할

In [ ]:
# (1) 학습용 데이터
churn_train = churn_parted[churn_parted['_PARTITION_'] == 0]

# (2) 평가용 데이터
churn_valid = churn_parted[churn_parted['_PARTITION_'] == 1]

# (3) 검증용 데이터
churn_test  = churn_parted[churn_parted['_PARTITION_'] == 2]

print("## Table Size (4:3:3)")
print(f" - Train Set: {churn_train.shape[0]:,}")
print(f" - Valid Set: {churn_valid.shape[0]:,}")
print(f" -  Test Set: {churn_test.shape[0]:,}")
print()
print("## Target Rate")
print(f" - Train Set: {churn_train['CHURN'].mean():.3f}")
print(f" - Valid Set: {churn_valid['CHURN'].mean():.3f}")
print(f" -  Test Set: {churn_test['CHURN'].mean():.3f}")

### **4. 변수 분리**  
- 입력변수, 타겟변수 분할

In [7]:
# (1) 전체
full_X = churn_train.drop(columns=['ID','CHURN','_PARTITION_'], errors='ignore')
full_y = churn_train['CHURN']

# (2) 학습용
X_train = churn_train.drop(columns=['ID','CHURN','_PARTITION_'], errors='ignore')
y_train = churn_train['CHURN']

# (3) 평가용
X_valid = churn_valid.drop(columns=['ID','CHURN','_PARTITION_'], errors='ignore')
y_valid = churn_valid['CHURN']

# (4) 검증용
X_test = churn_test.drop(columns=['ID','CHURN','_PARTITION_'], errors='ignore')
y_test = churn_test['CHURN']

### **4. 모형 적합**

#### **4.1. 의사결정나무**  
**(1) 모형 학습**

In [ ]:
dt = DecisionTreeClassifier(criterion = "chisquare",
                            max_depth = 10,
                            ccp_alpha = 0)

dt.fit(X_train, y_train)

**(2) 주요 Hyper Parameters**

- **criterion**: `{“chaid”, “chisquare”, “entropy|gain”, “igr|gainratio”, “gini”}`  
  각 트리 노드의 분할 기준을 지정합니다.

- **max_depth**: `int`  
  트리의 최대 깊이를 지정합니다.

- **ccp_alpha**: `float`  
  회귀 트리의 최소 비용 복잡성 가지치기에 사용할 값을 지정합니다.

<small> `DecisionTreeClassifier()` 클래스 사용에 대한 자세한 내용은 [**문서**](https://go.documentation.sas.com/doc/en/workbenchcdc/v_001/vwbpymlpg/sasviya-ml-tree-decisiontreeclassifier.htm)를 참고하세요.</small> 

#### **4.2. 랜덤포레스트**  
**(1) 모형 학습**

In [ ]:
#fit a random forest model
forest = ForestClassifier(n_estimators     = 100,
                          bootstrap        = 0.6,
                          criterion        = "chisquare",
                          max_depth        = 7,
                          min_samples_leaf = 5,
                          random_state     = 919)

forest.fit(X_train, y_train)

**(2) 주요 Hyper Parameters**

- **n_estimators**: `int`  
  생성할 트리의 수를 지정합니다.

- **bootstrap**: `float`  
  부트스트랩 샘플에 사용할 데이터의 비율을 지정합니다.

- **criterion**: `{“chaid”, “chisquare”, “entropy|gain”, “igr|gainratio”, “gini”}`  
  각 트리 노드에서 분할의 품질을 측정하는 기준을 지정합니다.

- **max_depth**: `int`  
  트리의 최대 깊이를 지정합니다.

- **min_samples_leaf**: `int`  
  각 노드에 최소한으로 있어야 하는 샘플의 수를 지정합니다.

- **random_state**: `int`  
  난수 생성에 사용할 시드를 지정합니다.

<small> `ForestClassifier()` 클래스 사용에 대한 자세한 내용은 [**문서**](https://go.documentation.sas.com/doc/en/workbenchcdc/v_001/vwbpymlpg/sasviya-ml-tree-forestclassifier.htm)를 참고하세요.</small> 

#### **4.3. 로지스틱회귀**  
**(1) 모형 학습**

In [ ]:
lr = LogisticRegression(tol       = 1e-4,
                        solver    = 'lbfgs',
                        selection = 'stepwise')

lr.fit(X_train, y_train)

**(2) 주요 Hyper Parameters**

- **tol**: `float`  
상대적인 기울기 수렴 기준을 지정합니다.

- **solver**: `str`  
사용할 최적화 알고리즘을 지정합니다. 유효한 옵션은 다음과 같습니다:
  - “congra”: 공액 기울기 방법
  - “dbldog”: 더블-도그레그 방법
  - “lbfgs”: 제한된 메모리 BFGS 해결기
  - “newrap”: 라인 탐색 및 리지법을 사용한 뉴턴-랩슨 방법
  - “nmsimp”: 넬더-미드 심플렉스 방법
  - “nrridg”: 리지법을 사용한 뉴턴-랩슨 방법
  - “quanew | duquanew”: 이중 준-뉴턴 최적화

- **selection**: `str`  
사용할 변수 선택 방법을 지정합니다. 유효한 옵션은 다음과 같습니다:
  - “backward”: 후진 선택법
  - “forward”: 전진 선택법
  - “lasso”: Lasso 방법
  - “stepwise”: 단계적 선택법

<small> `LogisticRegression()` 클래스 사용에 대한 자세한 내용은 [**문서**](https://go.documentation.sas.com/doc/en/workbenchcdc/v_001/vwbpymlpg/sasviya-ml-linear_model-logisticregression.htm)를 참고하세요.</small> 

#### **4.4. 지지벡터머신**  
**(1) 모형 학습**

In [ ]:
#fit a support vector machine classifier
svm = SVC(C     = 1.0,
          kernel= "rbf")

svm.fit(X_train, y_train)

**(2) 주요 Hyper Parameters**

- **C**: `float`  
비용 정규화 파라미터를 지정합니다.

- **kernel**: `{“linear”, “poly”, “polynomial”, “rbf”, “sigmoid”}`  
사용할 커널의 유형을 지정합니다.

- **random_state**: `int`  
난수 생성에 사용할 시드를 지정합니다.

<small> `SVC()` 클래스 사용에 대한 자세한 내용은 [**문서**](https://go.documentation.sas.com/doc/en/workbenchcdc/v_001/vwbpymlpg/sasviya-ml-svm-svc.htm)를 참고하세요.</small> 

#### **4.5. 그레디언트부스팅**  
**(1) 모형 학습**

In [ ]:
#fit a tree-based gradient boosting
gradboost = GradientBoostingClassifier(n_estimators     = 100,
                                       max_depth        = 4,
                                       min_samples_leaf = 5,
                                       learning_rate    = 0.1,
                                       subsample        = 0.8,
                                       random_state     = 919)

gradboost.fit(X_train, y_train)

**(2) 주요 Hyper Parameters**

- **n_estimators**: `int`  
  생성할 트리의 수를 지정합니다. 기본값은 100입니다.

- **min_samples_leaf**: `int`  
  각 노드에 최소한으로 있어야 하는 샘플의 수를 지정합니다. 기본값은 5입니다.

- **max_depth**: `int`  
  트리의 최대 깊이를 지정합니다. 기본값은 4입니다.

- **learning_rate**: `float`  
  각 트리의 학습률을 지정합니다. 기본값은 0.1입니다.

- **subsample**: `float`  
  각 트리를 구축하는 데 사용할 데이터의 비율을 지정합니다. 기본값은 0.5입니다.

- **random_state**: `float`  
  난수 생성기의 시드를 지정합니다. 기본값은 컴퓨터 시계를 기반으로 한 난수 스트림입니다. 재현 가능한 난수 시퀀스를 원할 경우 0보다 큰 값을 지정합니다.

<small> `GradientBoostingClassifier()` 클래스 사용에 대한 자세한 내용은 [**문서**](https://go.documentation.sas.com/doc/en/workbenchcdc/v_001/vwbpymlpg/sasviya-ml-tree-gradientboostingclassifier.htm)를 참고하세요.</small> 

### **5. 모형 평가**
- 모형 리스트 및 이름 정의

In [13]:
# 모델 리스트 및 이름 정의
models = [
    ("Decision Tree", "dt", dt),
    ("Random Forest", "rf", forest),
    ("Logistic Regression", "lr", lr),
    ("Support Vector Machine", "svm", svm),
    ("Gradient Boosting", "gbm", gradboost)
]

#### **5.1. 모형 평가 지표 산출**
- **주요 평가 지표**
  - **Accuracy(정확도)**: 전체 샘플 중 올바르게 분류된 비율
  - **Precision(정밀도)**: 해당 클래스에 대해 양성으로 예측된 것 중 실제 양성의 비율
  - **Recall(재현율)**: 실제 양성 중 모델이 양성으로 예측한 비율
  - **F1-Score(F1 점수)**: 정밀도와 재현율의 조화 평균 (데이터 불균형 시 유용)

In [15]:
# 결과 저장용 리스트
metrics = []
for name, abbr, model in models:
    # 학습용, 평가용, 테스트용 데이터 예측
    y_train_pred = model.predict(X_train)
    y_valid_pred = model.predict(X_valid)
    y_test_pred = model.predict(X_test)

    # 평가지표 계산
    accuracy_train = accuracy_score(y_train, y_train_pred)
    accuracy_valid = accuracy_score(y_valid, y_valid_pred)
    accuracy_test = accuracy_score(y_test, y_test_pred)

    precision_train = precision_score(y_train, y_train_pred, average="binary")
    precision_valid = precision_score(y_valid, y_valid_pred, average="binary")
    precision_test = precision_score(y_test, y_test_pred, average="binary")

    recall_train = recall_score(y_train, y_train_pred, average="binary")
    recall_valid = recall_score(y_valid, y_valid_pred, average="binary")
    recall_test = recall_score(y_test, y_test_pred, average="binary")

    f1_train = f1_score(y_train, y_train_pred, average="binary")
    f1_valid = f1_score(y_valid, y_valid_pred, average="binary")
    f1_test = f1_score(y_test, y_test_pred, average="binary")

    # 결과 저장
    metrics.append({
        "Model": name,
        "Accuracy_Train": round(accuracy_train, 3),
        #"Accuracy_Valid": round(accuracy_valid, 3),
        "Accuracy_Test": round(accuracy_test, 3),
        "Precision_Train": round(precision_train, 3),
        #"Precision_Valid": round(precision_valid, 3),
        "Precision_Test": round(precision_test, 3),
        "Recall_Train": round(recall_train, 3),
        #"Recall_Valid": round(recall_valid, 3),
        "Recall_Test": round(recall_test, 3),
        "F1_Score_Train": round(f1_train, 3),
        #"F1_Score_Valid": round(f1_valid, 3),
        "F1_Score_Test": round(f1_test, 3),
    })

# 결과 출력
metrics_df = pd.DataFrame(metrics)

In [ ]:
metrics_df

#### **5.2. ROC 그래프**

In [ ]:
# ROC 커브 그리기
plt.figure(figsize=(10, 8))

for name, abbr, model in models:
    # 학습용, 평가용, 테스트용 데이터 예측 확률 계산
    y_train_pred_proba = model.predict_proba(X_train).iloc[:, 1]
    y_valid_pred_proba = model.predict_proba(X_valid).iloc[:, 1]
    y_test_pred_proba = model.predict_proba(X_test).iloc[:, 1]

    # ROC 커브 계산
    fpr_train, tpr_train, _ = roc_curve(y_train, y_train_pred_proba)
    auc_train = auc(fpr_train, tpr_train)

    fpr_valid, tpr_valid, _ = roc_curve(y_valid, y_valid_pred_proba)
    auc_valid = auc(fpr_valid, tpr_valid)

    fpr_test, tpr_test, _ = roc_curve(y_test, y_test_pred_proba)
    auc_test = auc(fpr_test, tpr_test)

    # ROC 커브 그리기
    plt.plot(fpr_train, tpr_train, label=f"{name} (Train AUC = {auc_test:.3f})")
    #plt.plot(fpr_valid, tpr_valid, label=f"{name} (Valid AUC = {auc_valid:.3f})")
    plt.plot(fpr_test, tpr_test, label=f"{name} (Test AUC = {auc_test:.3f})")
        

plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random Guessing")  # 랜덤 분류 기준선
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend(loc="lower right")
plt.show()

#### **5.3. 향상도 그래프**

In [ ]:
# Lift 차트 그리기
plt.figure(figsize=(10, 8))

for name, abbr, model in models:
    # 학습용, 평가용, 테스트용 데이터 예측 확률 계산
    y_train_pred_proba = model.predict_proba(X_train).iloc[:, 1]
    y_valid_pred_proba = model.predict_proba(X_valid).iloc[:, 1]
    y_test_pred_proba = model.predict_proba(X_test).iloc[:, 1]

    # 전체 양성 비율 계산
    train_base_rate = np.mean(y_train)
    valid_base_rate = np.mean(y_valid)
    test_base_rate = np.mean(y_test)

    # 실제 값과 예측 확률을 결합하여 정렬
    train_data = np.array(list(zip(y_train, y_train_pred_proba)))
    train_data = train_data[train_data[:, 1].argsort()[::-1]]

    valid_data = np.array(list(zip(y_valid, y_valid_pred_proba)))
    valid_data = valid_data[valid_data[:, 1].argsort()[::-1]]

    test_data = np.array(list(zip(y_test, y_test_pred_proba)))
    test_data = test_data[test_data[:, 1].argsort()[::-1]]

    # 누적 양성 비율 계산
    train_cum_positives = np.cumsum(train_data[:, 0])
    valid_cum_positives = np.cumsum(valid_data[:, 0])
    test_cum_positives = np.cumsum(test_data[:, 0])

    # Lift 계산
    train_lift = train_cum_positives / (np.arange(1, len(train_cum_positives) + 1)) / train_base_rate
    valid_lift = valid_cum_positives / (np.arange(1, len(valid_cum_positives) + 1)) / valid_base_rate
    test_lift = test_cum_positives / (np.arange(1, len(test_cum_positives) + 1)) / test_base_rate

    # Lift 차트 그리기
    plt.plot(np.arange(1, len(train_lift) + 1) / len(train_lift), train_lift, label=f"{name} (Train Lift)")
    #plt.plot(np.arange(1, len(valid_lift) + 1) / len(valid_lift), valid_lift, label=f"{name} (Valid Lift)")
    plt.plot(np.arange(1, len(test_lift) + 1) / len(test_lift), test_lift, label=f"{name} (Test Lift)")

# 랜덤 추측 기준선
plt.plot([0, 1], [1, 1], linestyle="--", color="gray", label="Random Guessing")

plt.xlabel("Percent of Samples")
plt.ylabel("Lift")
plt.title("Lift Chart Comparison")
plt.legend(loc="upper right")
plt.show()


### **6. 스코어링**

In [20]:
# 각 모델에 대해 예측값과 확률 계산 후 결과 저장
for name, abbr, model in models:
    y_pred = pd.DataFrame(model.predict(full_X), columns=['I_CHURN'])
    y_pred_proba = pd.DataFrame(model.predict_proba(full_X), columns=['P_CHURN0', 'P_CHURN1'])
    result = pd.concat([churn_train[['ID', '_PARTITION_']], full_X, full_y, y_pred, y_pred_proba], axis=1)
    # 결과 데이터프레임명을 모델 이름으로 설정
    globals()[f"churn_{abbr}_score"] = result

### **7. 결과 저장**

In [ ]:
sas.df2sd(churn_dt_score, table='churn_py_dtr', libref='WRKLIB')
sas.df2sd(churn_rf_score, table='churn_py_rfm', libref='WRKLIB')
sas.df2sd(churn_lr_score, table='churn_py_log', libref='WRKLIB')
sas.df2sd(churn_svm_score, table='churn_py_svm', libref='WRKLIB')
sas.df2sd(churn_gbm_score, table='churn_py_gbm', libref='WRKLIB')

### **8. SAS 연결 종료**

In [ ]:
sas.endsas()